# **INITITALIZE**
Loading dataset to dataframe
Unzipping

In [1]:
#Mount drive for importing files from drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os
import zipfile
import cv2


In [3]:
#loading SCIN Spreadsheet into df
scinSpreadsheet_path = '/content/drive/Shareddrives/Skinterest-2b/data/SCIN Working Merged Spreadsheet.csv'
df = pd.read_csv(scinSpreadsheet_path)
df.head()


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,...,related_category,condition_duration,image_1_path,image_2_path,image_3_path,image_1_shot_type,image_2_shot_type,image_3_shot_type,combined_race,race_ethnicity_two_or_more_after_mitigation
0,-1.262950e+18,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,FST3,0,0,...,NaN,NaN,dataset/images/-5949315841433628424.png,dataset/images/-8183947049312687778.png,dataset/images/4923188439600899486.png,AT_DISTANCE,CLOSE_UP,AT_AN_ANGLE,NaN,NaN
1,-1.489550e+18,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,FST2,0,0,...,NaN,NaN,dataset/images/325464533153467313.png,NaN,NaN,CLOSE_UP,NaN,NaN,NaN,NaN
2,-1.492320e+18,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,NONE_IDENTIFIED,FST2,FST2,FST2,...,LOOKS_HEALTHY,ONE_TO_FOUR_WEEKS,dataset/images/-6837240536182868524.png,dataset/images/6395257111195214043.png,dataset/images/7877276387406078156.png,AT_DISTANCE,CLOSE_UP,AT_AN_ANGLE,HISPANIC_LATINO_OR_SPANISH_ORIGIN,NaN
3,-1.580900e+17,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST3,FST3,0,0,...,RASH,ONE_TO_FOUR_WEEKS,dataset/images/2983323875335943836.png,NaN,NaN,CLOSE_UP,NaN,NaN,WHITE,NaN
4,-1.887630e+18,SCIN,1.0.0,2023,AGE_30_TO_39,MALE,FST3,FST2,0,0,...,OTHER_ISSUE_DESCRIPTION,LESS_THAN_ONE_WEEK,dataset/images/3104801012387799539.png,dataset/images/8660513260658813359.png,NaN,CLOSE_UP,AT_AN_ANGLE,NaN,WHITE,NaN


In [ ]:
#unzip .zip files from folder and put image files into new 'image' folder

zip_drive_path = '/content/drive/Shareddrives/Skinterest-2b/data'
extraction_path = '/content/drive/Shareddrives/Skinterest-2b/data/ResNet50-images'
os.makedirs(extraction_path, exist_ok=True)

for file_name in os.listdir(zip_drive_path):
    if(file_name.endswith('.zip')):
        zip_path = os.path.join(zip_drive_path, file_name)
        with zipfile.ZipFile(zip_path, 'r') as zip:
            zip.extractall(extraction_path)
            print(f'Extracted {file_name}')

Extracted split_images_2.zip
Extracted split_images_4.zip
Extracted split_images_3.zip
Extracted split_images_1.zip
Extracted split_images_7.zip
Extracted split_images_5.zip
Extracted split_images_8.zip
Extracted split_images_6.zip


In [4]:
import matplotlib.pyplot as plt

# img = plt.imread('/content/drive/MyDrive/SQ BTT Data Playground/images/-6701195192556380.png')
# plt.imshow(img)
# plt.show()

In [5]:
data_dir = '/content/drive/Shareddrives/Skinterest-2b/data/ResNet50-images'
len(os.listdir(data_dir))

627

# **Image Quality Data Cleaning**
Removing blurry, corrupted, unusable images from SCIN database

In [6]:
from PIL import Image

In [7]:
!pip install opencv-python

In [ ]:
corrupted_files = []
duplicate_files = []


for file in os.listdir(data_dir):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(data_dir, file)
        # Corruption Test
        try:
            img = Image.open(file_path)
            img.verify()
            img = Image.open(file_path)
            img.load()
        except Exception as e:
            corrupted_files.append(file_path)

print("Corrupted files found: " + str(len(corrupted_files)))

Corrupted files found: 0


In [ ]:
BLUR_THRESHOLD = 25.0 # Adjustable, higher for better pics
blurry_files = []

for file in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file)
    # Blurriness Test
    img_grayscl = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    lap_variance = cv2.Laplacian(img_grayscl, cv2.CV_64F).var() # higher var = higer frq => better
    if lap_variance < BLUR_THRESHOLD:
        # print(f"Blurry file: {file} (Variance: {lap_variance:.2f})")
        blurry_files.append(file_path)

print("Blurry files found: " + str(len(blurry_files)))

Blurry files found: 143


In [9]:
!pip install imagehash --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.5 MB/s eta 0:00:00


In [10]:
import imagehash
import shutil

In [11]:
TRASH = '/content/drive/Shareddrives/Skinterest-2b/data/ResNet50_trash'
os.makedirs(TRASH, exist_ok=True)

In [ ]:
for path in blurry_files:
    dest = os.path.join(TRASH, os.path.basename(path))
    shutil.move(path, dest)

In [12]:
len(os.listdir(data_dir)) + len(os.listdir(TRASH))

770

In [ ]:
HASH_SIZE = 8

hash_keeper = {} # {hash_value: file_path_of_first_instance}
duplicate_files = []

for file in os.listdir(data_dir):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(data_dir, file)
        try:
            img = Image.open(file_path)
            img.load()
            p_hash = str(imagehash.phash(img, hash_size=HASH_SIZE))

            if p_hash in hash_keeper:
                original_path = hash_keeper[p_hash]
                duplicate_files.append((file_path, f"duplicate_of:{os.path.basename(original_path)}"))
            else:
                hash_keeper[p_hash] = file_path

        except Exception as e:
            print(f"File {file} has error: {e}")

In [ ]:
# DARK_THRESH = 75 # Image is too dark if the average pixel value is below this.
# BRIGHT_THRESH = 220 # Image is too bright if the average pixel value is above this.

# def is_extreme_lighting(path, dark_thresh=DARK_THRESH, bright_thresh=BRIGHT_THRESH):
#     """Checks if an image is over- or under-exposed."""

#     img_grayscl = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#     if img_grayscl is None:
#         return "load failed"

#     mean_intensity = np.mean(img_grayscl)
#     # print(mean_intensity)
#     if mean_intensity < dark_thresh:
#         return "dark"
#     elif mean_intensity > bright_thresh:
#         return "bright"
#     else:
#         return "good"

In [ ]:
# dark = []
# bright = []
# for file in os.listdir(data_dir):
#     file_path = os.path.join(data_dir, file)
#     img = Image.open(file_path)
#     width, height = img.size

#     lighting = is_extreme_lighting(file_path)
#     if lighting == "dark":
#         dark.append(file_path)
#     elif lighting == "bright":
#         bright.append(file_path)

In [ ]:
# len(dark)
# # len(bright)
# # len(failed)

In [ ]:
# for path in dark:
#     dest = os.path.join(TRASH, os.path.basename(path))
#     shutil.move(path, dest)

In [ ]:
TARGET_SIZE = (224, 224)
counter = 0

for file in os.listdir(data_dir):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(data_dir, file)
        img = Image.open(file_path)
        resized_img = img.resize(TARGET_SIZE, Image.Resampling.LANCZOS)
        resized_img.save(file_path)
        counter+=1

print(counter)


627


In [13]:
import cv2
import numpy as np

def segment_skin_hsv(img_hsv):
    """
    Creates a binary mask isolating skin pixels using predefined HSV bounds.
    """

    # Lower bound for skin (covering darker, less saturated tones)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)

    # Upper bound for skin (covering lighter, more saturated tones)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create a mask using the inRange function
    skin_mask = cv2.inRange(img_hsv, lower_skin, upper_skin)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    skin_mask = cv2.erode(skin_mask, kernel, iterations=1)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=1)

    return skin_mask

In [ ]:
# processed_images = {} # Dictionary to store the final processed NumPy arrays

# for file_name in os.listdir(data_dir):
#     if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
#         file_path = os.path.join(data_dir, file_name)

#         try:
#             img_bgr = cv2.imread(file_path)

#             # Color Space Transformation
#             img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV) # Convert to HSV
#             img_lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB) # Convert to LAB

#             # Skin Segmentation & ROI Extraction
#             skin_mask = segment_skin_hsv(img_hsv)

#             # Normalization (0-255 to 0.0-1.0)
#             normalized_img = img_bgr.astype('float32') / 255.0


#             processed_images[file_name] = {
#                 'hsv_array': img_hsv,
#                 'lab_array': img_lab,
#                 'rgb_normalized': normalized_img,
#                 'skin_mask': skin_mask
#             }

#         except Exception as e:
#             print(f"Failed to process {file_name}: {e}")

In [ ]:
# len(processed_images)

In [14]:
SHADOW_L_THRESH = 50  # Pixels darker than this are considered shadows
HIGHLIGHT_L_THRESH = 220 # Pixels brighter than this are considered highlights

def extract_lighting_features(img_array, skin_mask):
    """
    Calculates mean(brightness), std dev(contrast), and shadow/highlight ratios from the L* channel only from skin pixels defined by skin_mask
    """

    L_channel = img_array[:,:,0] # L* channel is index 0 in LAB
    L_skin = L_channel[skin_mask > 0]
    if L_skin.size == 0:
        return {
            'mean_L': 0.0,
            'std_L': 0.0,
            'shadow_ratio': 1.0,  # Treat as fully shadowed if no skin detected
            'highlight_ratio': 0.0
        }

    total_pixels = L_skin.size

    # Mean and Std Dev (Contrast)
    mean_luminosity = np.mean(L_skin) # higher = brighter image
    std_dev_luminosity = np.std(L_skin)

    # Shadow/Highlights Ratio
    shadow_count = np.sum(L_skin < SHADOW_L_THRESH)
    shadow_ratio = shadow_count / total_pixels

    highlight_count = np.sum(L_skin > HIGHLIGHT_L_THRESH)
    highlight_ratio = highlight_count / total_pixels

    features = {
        'mean_L': mean_luminosity,
        'std_L': std_dev_luminosity,
        'shadow_ratio': shadow_ratio,
        'highlight_ratio': highlight_ratio
    }
    return features

In [15]:
# Create training labels
MIN_CONTRAST_STD = 20.0 # STD below this indicates poor lighting
MAX_SHADOW_RATIO = 0.08 # More than this% of the image in harsh shadow
MAX_HIGHLIGHT_RATIO = 0.12 # More than this% of the image in harsh highlight

def assign_lighting_label(features):
    """Assigns a shadow label based on extracted features."""

    if features['highlight_ratio'] > MAX_HIGHLIGHT_RATIO:
        return 'overexposed'
    if features['shadow_ratio'] > MAX_SHADOW_RATIO:
        return 'harsh_shadow'
    if features['std_L'] < MIN_CONTRAST_STD:
        return 'poor_contrast'
    if features['mean_L'] < 50.0 and features['shadow_ratio'] > 0.05:
         return 'underexposed'
    return 'good'

In [ ]:
# file_path = '/content/drive/MyDrive/SQ BTT Data Playground/testfolder/test4.jpeg'

# processed_test = {}
# img_bgr = cv2.imread(file_path)
# img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV) # Convert to HSV
# img_lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB) # Convert to LAB
# normalized_img = img_bgr.astype('float32') / 255.0
# processed_test[file_name] = {
#         'hsv_array': img_hsv,
#         'lab_array': img_lab,
#         'rgb_normalized': normalized_img
# }

# features = extract_lighting_features(img_lab)
# label = assign_lighting_label(features)

# img = plt.imread(file_path)
# plt.imshow(img)
# plt.show()
# print(label)
# # processed_test

In [16]:
import random

def augment_lighting_undertone(img_lab_array):
    """Applies random brightness/contrast and a color temperature shift."""

    # 1. Lighting (Contrast/Brightness) Adjustment using Alpha/Beta (Simple form)
    # alpha (contrast) factor: 0.8 to 1.2
    alpha = random.uniform(0.8, 1.2)
    # beta (brightness) factor: -10 to +10
    beta = random.uniform(-10, 10)

    # Applies img_out = alpha * img_in + beta
    # Uses np.clip to keep pixel values within the 0-255 range
    img_aug = np.clip(alpha * img_lab_array + beta, 0, 255).astype(np.uint8)

    # 2. Undertone (Color Temperature) Shift in LAB space
    # LAB channels are L*(0), a*(1), b*(2). We shift b* (Blue-Yellow axis).
    # Shift between -8 (more blue/cool) and +8 (more yellow/warm)
    b_shift = random.uniform(-8, 8)

    # Apply shift only to the b* channel (index 2)
    img_aug[:,:,2] = np.clip(img_aug[:,:,2] + b_shift, 0, 255)

    # Convert the LAB array back to BGR for saving or final use
    img_bgr_aug = cv2.cvtColor(img_aug, cv2.COLOR_LAB2BGR)

    return img_bgr_aug

In [18]:
import random
import os
import cv2
import numpy as np


In [17]:

AUGMENTATION_COUNT = 4
AUG_DIR = '/content/drive/Shareddrives/Skinterest-2b/data/ResNet50-augmented_images'
os.makedirs(AUG_DIR, exist_ok=True)

for file_name in os.listdir(data_dir):
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(data_dir, file_name)

        try:
            # Load and convert to LAB
            img_bgr = cv2.imread(file_path)
            img_lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)

            original_file_name = file_name.split('.')[0]

            for i in range(AUGMENTATION_COUNT):
                # 1. Apply the augmentation
                img_bgr_augmented = augment_lighting_undertone(img_lab) # Use LAB array

                # 2. Create and save the new image
                aug_file_name = f'{original_file_name}_aug_{i}.png'
                aug_file_path = os.path.join(AUG_DIR, aug_file_name)
                cv2.imwrite(aug_file_path, img_bgr_augmented)

        except Exception as e:
            print(f"Skipping augmentation for {file_name}: {e}")

KeyboardInterrupt: 

In [19]:
import math
def calculate_ita_score(img_lab_array, skin_mask):
    """Calculates the mean Individual Typology Angle (ITA) score from skin pixels.

    ITA is a standard metric used to objectively classify skin color and tone,
    calculated from the L* (lightness) and b* (blue/yellow axis) channels.
    """

    # 1. Isolate L* and b* channels for skin pixels
    L_skin = img_lab_array[:,:,0][skin_mask > 0]
    b_skin = img_lab_array[:,:,2][skin_mask > 0]

    if L_skin.size == 0:
        return 0.0 # Handle case where no skin is detected

    # 2. Calculate mean values for the skin area
    mean_L = np.mean(L_skin)
    mean_b = np.mean(b_skin)

    # 3. Calculate ITA Score
    ita = (180.0 / np.pi) * math.atan((mean_L - 50.0) / mean_b)

    return ita

def assign_undertone_label(ita_score):
    """Assigns a categorical undertone label based on ITA score."""

    # Standardized thresholds for Cool, Neutral, and Warm undertones.
    if ita_score < -15:
        return 'cool'
    elif ita_score > 30:
        return 'warm'
    else:
        return 'neutral'

In [20]:
image_directories = [data_dir, AUG_DIR]

final_metadata = []

for current_dir in image_directories:
    for file_name in os.listdir(current_dir):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(current_dir, file_name)

            try:
                # 1. Load image and convert to LAB AND HSV
                img_bgr = cv2.imread(file_path)
                img_lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
                img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

                # 2. Skin Segmentation (Must happen first)
                skin_mask = segment_skin_hsv(img_hsv)

                # 3. GOAL 1: Lighting Feature Extraction & Labeling
                features = extract_lighting_features(img_lab, skin_mask)
                lighting_label = assign_lighting_label(features)

                # 4. GOAL 2: Undertone Feature Extraction & Labeling ⬅️ NEW CODE
                ita_score = calculate_ita_score(img_lab, skin_mask) # Calculate ITA
                undertone_label = assign_undertone_label(ita_score) # Assign label ('cool', 'warm', 'neutral')

                # 5. Store result
                final_metadata.append({
                    'image_path': file_path,
                    'lighting_label': lighting_label,
                    'undertone_label': undertone_label,
                    'ita_score': ita_score,

                    'mean_L': features['mean_L'],
                    'std_L': features['std_L'],
                    'shadow_ratio': features['shadow_ratio'],
                    'highlight_ratio': features['highlight_ratio']
                })

            except Exception as e:
                print(f"Skipping {file_name} in {current_dir}: {e}")


df_final = pd.DataFrame(final_metadata)
df_final.to_csv('/content/drive/Shareddrives/Skinterest-2b/data/ResNet50_FINAL_TRAINING_METADATA.csv', index=False)
print(f"\nFinal training metadata created with {len(df_final)} entries.")


Final training metadata created with 3135 entries.


In [21]:
len(os.listdir('/content/drive/Shareddrives/Skinterest-2b/data/ResNet50-augmented_images'))

2508

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder




In [23]:
df_final = pd.read_csv('/content/drive/Shareddrives/Skinterest-2b/data/ResNet50_FINAL_TRAINING_METADATA.csv')

X = df_final['image_path']
y = df_final[['lighting_label', 'undertone_label']]

# 1. Split into Training (80%) and a Test set (20%)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

print(f"Train image paths: {len(X_train)} | Test image paths: {len(X_test)}")

Train image paths: 2508 | Test image paths: 627


In [24]:
# Encoding df_final so that the labels are integers
lighting_enc = LabelEncoder()
undertone_enc = LabelEncoder()

all_light_labels = np.concatenate([Y_train['lighting_label'], Y_test['lighting_label']])
lighting_enc.fit(all_light_labels)
y1_train = lighting_enc.transform(Y_train['lighting_label'])
y1_test = lighting_enc.transform(Y_test['lighting_label'])

all_tone_labels = np.concatenate([Y_train['undertone_label'], Y_test['undertone_label']])
undertone_enc.fit(all_tone_labels)
y2_train = undertone_enc.transform(Y_train['undertone_label'])
y2_test = undertone_enc.transform(Y_test['undertone_label'])

NUM_LIGHT_CLASSES = len(lighting_enc.classes_)
NUM_TONE_CLASSES = len(undertone_enc.classes_)
print(f"Lighting Classes: {NUM_LIGHT_CLASSES} ({lighting_enc.classes_})")
print(f"Undertone Classes: {NUM_TONE_CLASSES} ({undertone_enc.classes_})")

Lighting Classes: 4 (['good' 'harsh_shadow' 'overexposed' 'poor_contrast'])
Undertone Classes: 2 (['neutral' 'warm'])


In [25]:
def multi_output_generator(X_data, y1_data, y2_data, batch_size, target_size=(224, 224)):
    """
    Custom generator that loads images from paths and yields a batch of images
    along with two separate target label arrays.
    """
    num_samples = len(X_data)

    # OHE
    y1_data_ohe = tf.keras.utils.to_categorical(y1_data, num_classes=NUM_LIGHT_CLASSES)
    y2_data_ohe = tf.keras.utils.to_categorical(y2_data, num_classes=NUM_TONE_CLASSES)

    i = 0
    while True:
        if i * batch_size >= num_samples:
            i = 0

        batch_indices = np.arange(i * batch_size, min((i + 1) * batch_size, num_samples))

        X_batch = np.zeros((len(batch_indices), target_size[0], target_size[1], 3), dtype=np.float32)
        y1_batch = y1_data_ohe[batch_indices] # Lighting
        y2_batch = y2_data_ohe[batch_indices] # Undertone

        for j, idx in enumerate(batch_indices):
            path = X_data.iloc[idx] # Grabs image path

            try:
                img = cv2.imread(path)

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


                # Normalize (0-255 to 0-1)
                img = img.astype('float32') / 255.0

                # Store in batch
                X_batch[j] = img

            except Exception as e:
                print(f"Error loading image {path}: {e}")

        i += 1
        # Yield the batch
        yield X_batch, (y1_batch, y2_batch)

In [26]:
input_tensor = Input(shape=(224, 224, 3), name='image_input')

base_model = ResNet50(weights='imagenet',
                      include_top=False,
                      input_tensor=input_tensor)

base_model.trainable = True

for layer in base_model.layers[:140]:
    layer.trainable = False


# Shared Feature Layer
shared_features = base_model.output
shared_features = GlobalAveragePooling2D()(shared_features)


# Head 1 (Lighting)
lighting_output = Dense(128, activation='relu')(shared_features)
lighting_output = Dense(NUM_LIGHT_CLASSES, activation='softmax', name='lighting_output')(lighting_output)


# Head 2 (Undertone)
undertone_output = Dense(128, activation='relu')(shared_features)
undertone_output = Dense(NUM_TONE_CLASSES, activation='softmax', name='undertone_output')(undertone_output)


# The Multi-Task Model
multi_task_model = Model(inputs=input_tensor,
                         outputs=[lighting_output, undertone_output])


# Compile the Model
multi_task_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    loss_weights={'lighting_output': 3, 'undertone_output': 1.0},

    loss={
        'lighting_output': 'categorical_crossentropy',
        'undertone_output': 'categorical_crossentropy'
    },
    metrics={
        'lighting_output': ['accuracy'],
        'undertone_output': ['accuracy']
    }
)

multi_task_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ image_input[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,113,030 (91.98 MB)

 Trainable params: 15,503,366 (59.14 MB)

 Non-trainable params: 8,609,664 (32.84 MB)

In [27]:
BATCH_SIZE = 32

EPOCHS = 20
STEPS_PER_EPOCH = len(X_train) // BATCH_SIZE
VALIDATION_STEPS = len(X_test) // BATCH_SIZE


output_types = (
    tf.float32,
    (tf.float32, tf.float32)
)

output_shapes = (
    tf.TensorShape([None, 224, 224, 3]),
    (tf.TensorShape([None, NUM_LIGHT_CLASSES]),
     tf.TensorShape([None, NUM_TONE_CLASSES]))
)


train_dataset = tf.data.Dataset.from_generator(
    lambda: multi_output_generator(X_train, y1_train, y2_train, BATCH_SIZE),
    output_types=output_types,
    output_shapes=output_shapes
).prefetch(tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: multi_output_generator(X_test, y1_test, y2_test, BATCH_SIZE),
    output_types=output_types,
    output_shapes=output_shapes
).prefetch(tf.data.AUTOTUNE)


# Train
history = multi_task_model.fit(
    train_dataset,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=validation_dataset,
    validation_steps=VALIDATION_STEPS
)

Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 55s 366ms/step - lighting_output_accuracy: 0.4764 - lighting_output_loss: 1.2524 - loss: 4.4312 - undertone_output_accuracy: 0.5722 - undertone_output_loss: 0.6738 - val_lighting_output_accuracy: 0.1053 - val_lighting_output_loss: 1.6150 - val_loss: 5.5550 - val_undertone_output_accuracy: 0.4161 - val_undertone_output_loss: 0.7100
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 32s 303ms/step - lighting_output_accuracy: 0.5622 - lighting_output_loss: 1.0882 - loss: 3.9054 - undertone_output_accuracy: 0.6358 - undertone_output_loss: 0.6381 - val_lighting_output_accuracy: 0.1086 - val_lighting_output_loss: 1.6404 - val_loss: 5.6039 - val_undertone_output_accuracy: 0.5526 - val_undertone_output_loss: 0.6828
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 24s 313ms/step - lighting_output_accuracy: 0.5729 - lighting_output_loss: 1.0411 - loss: 3.7375 - undertone_output_accuracy: 0.6896 - undertone_output_loss: 0.6115 - val_lighting_output_accuracy: 0.1793 - val_lighting_ou

In [ ]:
BATCH_SIZE = 32

EPOCHS = 20
STEPS_PER_EPOCH = len(X_train) // BATCH_SIZE
VALIDATION_STEPS = len(X_test) // BATCH_SIZE


output_types = (
    tf.float32,
    (tf.float32, tf.float32)
)

output_shapes = (
    tf.TensorShape([None, 224, 224, 3]),
    (tf.TensorShape([None, NUM_LIGHT_CLASSES]),
     tf.TensorShape([None, NUM_TONE_CLASSES]))
)


train_dataset = tf.data.Dataset.from_generator(
    lambda: multi_output_generator(X_train, y1_train, y2_train, BATCH_SIZE),
    output_types=output_types,
    output_shapes=output_shapes
).prefetch(tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: multi_output_generator(X_test, y1_test, y2_test, BATCH_SIZE),
    output_types=output_types,
    output_shapes=output_shapes
).prefetch(tf.data.AUTOTUNE)


# Train
history = multi_task_model.fit(
    train_dataset,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=validation_dataset,
    validation_steps=VALIDATION_STEPS
)

Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 50s 399ms/step - lighting_output_accuracy: 0.2420 - lighting_output_loss: 1.4332 - loss: 4.9950 - undertone_output_accuracy: 0.5000 - undertone_output_loss: 0.6954 - val_lighting_output_accuracy: 0.1102 - val_lighting_output_loss: 2.1078 - val_loss: 7.0078 - val_undertone_output_accuracy: 0.5806 - val_undertone_output_loss: 0.6844
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 38s 394ms/step - lighting_output_accuracy: 0.5535 - lighting_output_loss: 1.1092 - loss: 3.9859 - undertone_output_accuracy: 0.5834 - undertone_output_loss: 0.6690 - val_lighting_output_accuracy: 0.1382 - val_lighting_output_loss: 1.7916 - val_loss: 6.0498 - val_undertone_output_accuracy: 0.6283 - val_undertone_output_loss: 0.6751
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 25s 321ms/step - lighting_output_accuracy: 0.5763 - lighting_output_loss: 1.0492 - loss: 3.7860 - undertone_output_accuracy: 0.6305 - undertone_output_loss: 0.6491 - val_lighting_output_accuracy: 0.2039 - val_lighting_ou

In [29]:
multi_task_model.save('my_classifier_model.keras')

In [30]:
from google.colab import files

# This initiates a download dialog in your browser for the specified file
files.download('my_classifier_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>